# Clasificador Marvel.
¿Alguna vez has deseado saber el nombre de tus personajes favoritos de Marvel pero tienes una pésima memoria? Ya no vuelvas a pasar por momentos así de vergonzosos, pues el **Clasificador Marvel** ha llegado a salvar el día.  

Patrocinado por:
- Azure Cloud.
- Azure Cognitive Services: Custom Vision.
- Innovacción Microsoft.

## Ejecución.
Es muy sencillo de usar siempre y cuando sepas correr Jupyter Notebooks, o puedas correr el siguiente código de Python.  

El primer paso es instalar el cliente de Azure, con el siguiente comando:

In [ ]:
!pip install azure-cognitiveservices-vision-customvision

Luego importamos algunas bibliotecas que nos servirán para conectarnos con Custom Vision, desplegar imágenes, entre otras cosas:

In [ ]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline

Luego vamos a preparar algunas variables que necesitamos para ejecutar el código.  

Las variables `project_id`, `cv_key`, `cv_endpoint` y `model_name` lo obtenemos tras entrenar nuestro modelo de Custom Vision.

In [ ]:
# Set up
project_id = '5bfbfe4c-1feb-4d03-981f-6a5609fb964d'
cv_key = '1efbfa4470614f0b94e5b97f135c8363'
cv_endpoint = 'https://machinelearningwee-prediction.cognitiveservices.azure.com/'
model_name = 'Iteration1'

directory = 'data'
images = os.listdir(directory)

credentials = ApiKeyCredentials(in_headers={"Prediction-key": cv_key})
custom_vision_client = CustomVisionPredictionClient(endpoint=cv_endpoint, credentials=credentials)

Con lo anterior listo, ya podemos comenzar a clasificar. De esto se encarga el cliente de Custom Vision, sólo debemos indicarle la imagen a clasificar.

In [ ]:
#Classify
classification_results = []

for i in range(len(images)):
    print("Classifying image {} out of {}".format(i+1, len(images)))
    image_data = open(os.path.join(directory, images[i]), "rb")
    classification = custom_vision_client.classify_image(project_id, model_name, image_data.read())
    classification_results.append(classification)

Lo anterior puede tomar un par de segundos en ejecutarse.  

El siguiente código nos sirve para deplegar los resultados:

In [ ]:
fig = plt.figure(figsize=(32, 32))

for i in range(len(images)):
    prediction = classification_results[i].predictions[0].tag_name
    img = Image.open(os.path.join(directory, images[i]))

    a=fig.add_subplot(len(images)/4, 4,i+1)
    a.axis('off')
    imgplot = plt.imshow(img)
    a.set_title(prediction)

plt.show()

En nuestro experimento, obtuvimos 37 de 40 imágenes correctas, un 92.5%.  

Pero no nos detuvimos ahí...

## Logic Apps
Ya vimos que podemos consumir fácilmente el modelo de predicción. Pero a veces queremos usar esa información para mejorar nuestro modelo a futuro.  

Una idea que se nos ocurrió es mantener un registro de las imágenes que se clasifican y los resultados.  

Con ese propósito utilizamos un flujo del trabajo como el siguiente, todo en la nube de Azure:
![Imagen](/assets/IA-Reto3.jpeg)

Esto mismo se puede extender, de manera que un script, app, sitio web, etc, suba las imágenes al Blob Storage. Ya con la información almacenada en la nube, las posibilidades son ilimitadas y depende de nosotros los desarrolladores generar las ideas.

Además, consideramos que esta misma configuración se puede extender a otros proyectos, con otros modelos de Custom Vision.